In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
import keras_tuner as kt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [ ]:
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")


In [ ]:
PATH = '/kaggle/input/dog-breed-identification'
NUM_CHANNEL = 3
INPUT_SHAPE = 256
BATCH_SIZE = 32 * strategy.num_replicas_in_sync


In [ ]:
class ImageDatastore:
    
    def __init__(self, path, csv, output_shape, train_val_test):
        self.path = path
        self.csv = csv
        self.output_shape = output_shape
        self.train_val_test = train_val_test
        self.image_paths, self.labels = self.get_files_and_labels()
        
    def get_files_and_labels(self):
        image_paths = [os.path.join(self.path, path) + '.jpg' for path in self.csv.index]
        if self.train_val_test == 'test':
            labels = ['' for i in range(len(image_paths))]
        else:
            labels = pd.get_dummies(self.csv.breed).astype('uint8').to_numpy()
        return image_paths, labels
    
    def __call__(self):
        pairs = list(zip(self.image_paths, self.labels))
        for image_path, label in pairs:
            image = cv2.imread(image_path)
            image = cv2.resize(image, self.output_shape)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            if self.train_val_test == 'test':
                yield image
            else:
                yield image, label


In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, monitor="loss", factor=0.5, patience=0, min_lr=0.01):
        super(CustomCallback, self).__init__()
        self.monitor = monitor
        self.factor = factor
        self.patience = patience
        self.min_lr = min_lr
        self.job = 0

    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        if 'loss' in self.monitor:
            self.best = np.inf
        else:
            self.best = -1

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if 'loss' in self.monitor and current < self.best:
            self.best = current
            self.wait = 0
        elif 'acc' in self.monitor and  current > self.best:
            self.best = current
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                if self.job == 0:
                    lr = float(K.get_value(self.model.optimizer.learning_rate))
                    new_lr = lr * self.factor
                    if new_lr < self.min_lr:
                        new_lr = self.min_lr
                        self.job = 1
                    K.set_value(self.model.optimizer.lr, new_lr)
                    self.wait = 0
                    print(f"\nLearning rate reduced from {'{:.3g}'.format(lr)} to {'{:.3g}'.format(new_lr)}")
                elif self.job == 1:
                    self.stopped_epoch = epoch
                    self.model.stop_training = True

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print(f"Epoch {self.stopped_epoch + 1}: early stopping")


In [ ]:
train_df = pd.read_csv(os.path.join(PATH, 'labels.csv'), index_col='id')
train_df.head()


In [ ]:
train_df.value_counts().plot.pie(autopct='%%%.2f', figsize=(25, 50));


In [ ]:
samples = train_df.sample(6)
plt.figure(figsize=(12, 8))

for i, index in enumerate(samples.index):
    plt.subplot(230 + i + 1)
    img = mpimg.imread(os.path.join(PATH, 'train', index + '.jpg'))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'True Class: {samples.loc[[index], "breed"].values[0]}')
    
plt.tight_layout()
plt.show()


In [ ]:
NUM_CLASS = train_df.breed.nunique()
print(f'Number of classes: {NUM_CLASS}')


In [ ]:
val_ratio = 0.2
num_sapmle = int(len(train_df) * val_ratio / NUM_CLASS)


In [ ]:
val_df = pd.concat([train_df[train_df.breed == lbl].sample(num_sapmle) for lbl in train_df.breed.unique()], axis=0)
val_df = val_df.sample(frac=1)

train_df = train_df.drop(val_df.index)


In [ ]:
print(f'''Number of train images: {len(train_df)}
Number of val images: {len(val_df)}''')


In [ ]:
train_ds = ImageDatastore(os.path.join(PATH, 'train'), train_df, (INPUT_SHAPE, INPUT_SHAPE), 'train')
val_ds = ImageDatastore(os.path.join(PATH, 'train'), val_df, (INPUT_SHAPE, INPUT_SHAPE), 'val')


In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2)
])


In [ ]:
OUTPUT_SIGNATURE = (tf.TensorSpec(shape=(INPUT_SHAPE, INPUT_SHAPE, NUM_CHANNEL), dtype='uint8'), tf.TensorSpec(shape=(NUM_CLASS), dtype='uint8'))

train = tf.data.Dataset.from_generator(generator=train_ds, output_signature=OUTPUT_SIGNATURE)
train = tf.data.Dataset.range(1).interleave(lambda _: train, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size=BATCH_SIZE, drop_remainder=True).map(lambda X, y: (data_augmentation(X, training=True), y), num_parallel_calls=tf.data.AUTOTUNE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

val = tf.data.Dataset.from_generator(generator=val_ds, output_signature=OUTPUT_SIGNATURE)
val = tf.data.Dataset.range(1).interleave(lambda _: val, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size=BATCH_SIZE, drop_remainder=True).cache().prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
def build_model(hp):

    input_shape = (INPUT_SHAPE, INPUT_SHAPE, NUM_CHANNEL)

    base = tf.keras.applications.MobileNetV2(input_shape=input_shape, weights='imagenet', include_top=False, pooling='avg')
    base.trainable=False
    base.training=False

    inputs = tf.keras.Input(shape=input_shape)
    
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = base(x)
    
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    x=tf.keras.layers.Dense(hp_units)(x)
    x=tf.keras.layers.BatchNormalization()(x)
    
    hp_act = hp.Choice('activation', values=['relu', 'tanh'])
    x=tf.keras.layers.Activation(hp_act)(x)
    
    hp_drop = hp.Float('rate', min_value=0.0, max_value=0.5, step=0.1)
    x=tf.keras.layers.Dropout(hp_drop)(x)
    
    outputs = tf.keras.layers.Dense(NUM_CLASS, activation="softmax")(x)
    
    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
        metrics=[tf.keras.metrics.CategoricalAccuracy(name="accuracy")],
    )
    return model


In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    distribution_strategy=strategy)


In [ ]:
tuner.search(train, epochs=10, validation_data=val)


In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete.
The optimal number of units in the first densely-connected layer is {best_hps.get('units')}.
The optimal activation for the first densely-connected is {best_hps.get('activation')}.
The optimal rate for dropout is {best_hps.get('rate')}.
""")


In [ ]:
with strategy.scope():
    model = tuner.hypermodel.build(best_hps)

model.summary()


In [ ]:
history = model.fit(
    train,
    epochs=1000,
    validation_data=val,
    callbacks=[CustomCallback(monitor='val_accuracy', factor=0.5, patience=10, min_lr=2e-6)]
)


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


In [ ]:
classes = sorted(train_df.breed.unique())
print(classes)


In [ ]:
plt.figure(figsize=(12, 8))

for i, index in enumerate(samples.index):
    img = mpimg.imread(os.path.join(PATH, 'train', index + '.jpg'))
    
    img = cv2.resize(img, (INPUT_SHAPE, INPUT_SHAPE))
    pred = model.predict(np.expand_dims(img, 0), verbose=False)
    cls_index = np.argmax(pred)
    cls_name = classes[cls_index]
    
    plt.subplot(230 + i + 1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'True Class: {samples.loc[[index], "breed"].values[0]} \n Predicted Class: {cls_name}')
    
plt.tight_layout()
plt.show()


In [ ]:
sub_test_df = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv', index_col='id')


In [ ]:
sub_test_ds = ImageDatastore(os.path.join(PATH, 'test'), sub_test_df, (INPUT_SHAPE, INPUT_SHAPE), 'test')


In [ ]:
SUB_OUTPUT_SIGNATURE = (tf.TensorSpec(shape=(INPUT_SHAPE, INPUT_SHAPE, NUM_CHANNEL), dtype='uint8'))
sub_test = tf.data.Dataset.from_generator(generator=sub_test_ds, output_signature=SUB_OUTPUT_SIGNATURE)
sub_test = tf.data.Dataset.range(1).interleave(lambda _:sub_test, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size=BATCH_SIZE, drop_remainder=False).cache().prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
pred = model.predict(sub_test)


In [ ]:
sub_test_df.iloc[:] = pred


In [ ]:
sub_test_df.to_csv(os.path.join('/kaggle', 'working', 'submission.csv'))
sub_test_df.head()
